# MSA 2023
https://github.com/xylarshayu

---

1. MELD - Dataset that's been used. https://github.com/declare-lab/MELD
2. Previous attempt (along with data analysis on dataset): https://colab.research.google.com/drive/1MYaJZZshcTsmVipJCqeHbdDpmX59QfmG?usp=sharing
3. 


## Some knobs and dials for the notebook

In [1]:
DOWNLOAD_DATASET_FROM_SOURCE = False # Downloading the entire dataset from original source instead of just zip file from my drive


## Getting the Dataset

In [2]:
!pip install gdown transformers tqdm speechbrain optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 71.6 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.0/519.0 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.1/757.1 kB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 102.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 65.8 MB/s eta 0:00:00

In [2]:
import os
import requests
import tarfile
import shutil
import zipfile
import gdown
import subprocess

In [4]:
def download_and_extract_meld_dataset():
  subprocess.run(["wget", "https://web.eecs.umich.edu/~mihalcea/downloads/MELD.Raw.tar.gz"])

  subprocess.run(["tar", "-xzvpf", "MELD.Raw.tar.gz", "-C", "."])
  subprocess.run(["tar", "-xzvf", "MELD.Raw/dev.tar.gz", "-C", "MELD.Raw/"])
  subprocess.run(["tar", "-xzvf", "MELD.Raw/test.tar.gz", "-C", "MELD.Raw/"])
  subprocess.run(["tar", "-xzvf", "MELD.Raw/train.tar.gz", "-C", "MELD.Raw/"])

  subprocess.run(["rm", "MELD.Raw.tar.gz"])
  subprocess.run(["rm", "MELD.Raw/dev.tar.gz"])
  subprocess.run(["rm", "MELD.Raw/test.tar.gz"])
  subprocess.run(["rm", "MELD.Raw/train.tar.gz"])

def download_meld_zip():
  url = "https://drive.google.com/uc?id=11vTa_5JYNQmwM9F4dPXHMFBFoFXsRYfC"
  gdown.download(url, output="MELD.Raw.zip", quiet=False)
  meld_zip_file = "MELD.Raw.zip"
  with zipfile.ZipFile(meld_zip_file, 'r') as zip_ref:
      zip_ref.extractall()
  shutil.move("content/MELD.Raw", ".")
  os.remove(meld_zip_file)
  shutil.rmtree("content")

# Check if the MELD dataset already exists in the current directory
if not os.path.isdir("MELD.Raw"):
    download_and_extract_meld_dataset() if DOWNLOAD_DATASET_FROM_SOURCE else download_meld_zip()


Downloading...
From (uriginal): https://drive.google.com/uc?id=11vTa_5JYNQmwM9F4dPXHMFBFoFXsRYfC
From (redirected): https://drive.google.com/uc?id=11vTa_5JYNQmwM9F4dPXHMFBFoFXsRYfC&confirm=t&uuid=349836c4-0cfe-4a2f-a07b-c09cbb24c36b
To: /home/jupyter/MELD.Raw.zip
100%|██████████| 10.9G/10.9G [00:43<00:00, 251MB/s]


## Data Preperation

1. Importing CSV files
2. Converting audio to embeddings using SpeechBrain
3. Converting Text to embeddings using BERT
4. Saving these neatly to drive as checkpoints

In [19]:
!pip uninstall torch torchvision torchaudio -y
!pip install --no-cache-dir torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Found existing installation: torch 1.10.0+cu113
Uninstalling torch-1.10.0+cu113:
  Successfully uninstalled torch-1.10.0+cu113
Found existing installation: torchvision 0.11.1+cu113
Uninstalling torchvision-0.11.1+cu113:
  Successfully uninstalled torchvision-0.11.1+cu113
Found existing installation: torchaudio 0.10.0+cu113
Uninstalling torchaudio-0.10.0+cu113:
  Successfully uninstalled torchaudio-0.10.0+cu113
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 34.1 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 758.6 kB/s eta 0:00:00a 0:00:01


In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pickle
import datetime
import math
from tqdm import tqdm
import sys
import multiprocessing
from transformers import BertTokenizer
from speechbrain.pretrained import EncoderClassifier
from concurrent.futures import ProcessPoolExecutor
from sklearn.preprocessing import LabelEncoder

num_cores = multiprocessing.cpu_count()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
SB_model = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_models/spkrec-ecapa-voxceleb")

### Importing csv files

In [3]:
train_df = pd.read_csv('MELD.Raw/train_sent_emo.csv')
dev_df = pd.read_csv('MELD.Raw/dev_sent_emo.csv')
test_df = pd.read_csv('MELD.Raw/test_sent_emo.csv')

In [4]:
#@title Remove all rows where the duration is greater than 5 seconds
train_df['StartTime'] = pd.to_datetime(train_df['StartTime'])
train_df['EndTime'] = pd.to_datetime(train_df['EndTime'])
test_df['StartTime'] = pd.to_datetime(test_df['StartTime'])
test_df['EndTime'] = pd.to_datetime(test_df['EndTime'])
dev_df['StartTime'] = pd.to_datetime(dev_df['StartTime'])
dev_df['EndTime'] = pd.to_datetime(dev_df['EndTime'])

# Calculate the duration of each utterance as the difference between the 'EndTime' and 'StartTime' values
train_df['Duration'] = train_df['EndTime'] - train_df['StartTime']
test_df['Duration'] = test_df['EndTime'] - test_df['StartTime']
dev_df['Duration'] = dev_df['EndTime'] - dev_df['StartTime']
# Convert the 'Duration' column to seconds
train_df['Duration'] = train_df['Duration'].dt.total_seconds()
test_df['Duration'] = test_df['Duration'].dt.total_seconds()
dev_df['Duration'] = dev_df['Duration'].dt.total_seconds()
# Remove all rows where the duration is greater than 5 seconds
train_df = train_df.drop(train_df[train_df['Duration'] >= 5].index)
test_df = test_df.drop(test_df[test_df['Duration'] >= 5].index)
dev_df = dev_df.drop(dev_df[dev_df['Duration'] >= 5].index)

dataframes = { 'train': {'df':train_df, 'folder':'train_splits'}, 'dev': {'df':dev_df, 'folder':'dev_splits_complete'}, 'test': {'df': test_df, 'folder': 'output_repeated_splits_test'}}

In [4]:
# Adding Colored Printing
red = '\033[91m'
green = '\033[92m'
yellow = '\033[93m'
blue = '\033[94m'
pink = '\033[95m'
teal = '\033[96m'
grey = '\033[97m'
print(teal, "testing how to print colored text")

 testing how to print colored text


### Extracting features from data

In [7]:
!pip install moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 62.0 MB/s eta 0:00:0000:0100:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110730 sha256=8993e559f5229b55932c12cdaceef47f9a966cdf246712f65a118dca113ce92d
  Stored in directory: /home/jupyter/.cache/pip/wheels/56/dc/2b/9cd600d483c04af3353d66623056fc03faed76b7518faae4df
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1


In [6]:
from moviepy.editor import AudioFileClip

data_dict_train = {}
data_dict_dev = {}
data_dict_test = {}
data_dicts = {'train': data_dict_train, 'dev': data_dict_dev, 'test': data_dict_test}

sentiment_encoder = LabelEncoder()
emotion_encoder = LabelEncoder()
sentiment_encoder.fit(train_df['Sentiment'])
emotion_encoder.fit(train_df['Emotion'])

LabelEncoder()

In [10]:
# Processes single data sample
def process_file(dataset, sr_no):
    success = True
    folder_name = dataframes[dataset]['folder']
    df = dataframes[dataset]['df']
    dialogue_id, utterance_id = df.loc[sr_no, 'Dialogue_ID'], df.loc[sr_no, 'Utterance_ID']
    file_path = f"MELD.Raw/{folder_name}/dia{dialogue_id}_utt{utterance_id}.mp4"
    raw_text = df.loc[sr_no, 'Utterance']

    try:
      raw_audio = AudioFileClip(file_path).to_soundarray(fps=22000)
      if raw_audio.shape[1] > 1:
            raw_audio = raw_audio.mean(axis=1)
      raw_audio = torch.tensor(raw_audio).unsqueeze(0)
    except Exception as e:
      print(yellow, "Error loading file ", file_path)
      print(red, type(e), ":\n", e)
      success = False
      return success, None, None, None, None
    
    audio_features = SB_model.encode_batch(raw_audio).mean(dim=1).squeeze(0)
    text_features = tokenizer(raw_text, padding='max_length', truncation=True, max_length=256, return_tensors="pt")
    sentiment_label = sentiment_encoder.transform([df.loc[sr_no, 'Sentiment']])[0]
    emotion_label = emotion_encoder.transform([df.loc[sr_no, 'Emotion']])[0]
    
    return success, audio_features, text_features, sentiment_label, emotion_label

In [7]:
# Helper function to run process_file in parallel
import funcs

def process_files_in_parallel(dataset):
    df = dataframes[dataset]['df']
    folder_name = dataframes[dataset]['folder']
    successful_indices = []
    
    print(teal, "Processing files in parallel")
    if __name__ == '__main__':
        with ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
            pbar = tqdm(total=df.shape[0])
            for sr_no in df.index:
                print(yellow, "Processing file ", sr_no)
                
                success, raw_audio, text_features, sentiment_label, emotion_label = executor.submit(funcs.process_file, folder_name, df, sr_no, sentiment_encoder, emotion_encoder, tokenizer).result()
                if success:
                    audio_features = SB_model.encode_batch(raw_audio).mean(dim=1).squeeze(0)
                    print(blue, "Audio features encoded")
                    data_dicts[dataset][sr_no] = {'audio': audio_features, 'text': text_features, 'sentiment': sentiment_label, 'emotion': emotion_label}
                    successful_indices.append(sr_no)
                pbar.update()
    
    pbar.close()
    dataframes[dataset]['df'] = df.loc[successful_indices]

    # Save checkpoints
    df.to_csv(f'updated_{dataset}_df.csv', index=False)
    with open(f'data_dict_{dataset}.pickle', 'wb') as handle:
        pickle.dump(data_dicts[dataset], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
def process_files_sequentially(dataset):
    df = dataframes[dataset]['df']
    successful_indices = []
    print(teal, "Processing files in parallel")
    for sr_no in tqdm(df.index, desc=f"Processing {dataset}"):
        success, audio_features, text_features, sentiment_label, emotion_label = process_file(dataset, sr_no)
        if success:
            data_dicts[dataset][sr_no] = {'audio': audio_features, 'text': text_features, 'sentiment': sentiment_label, 'emotion': emotion_label}
            successful_indices.append(sr_no)
    
    dataframes[dataset]['df'] = df.loc[successful_indices]

    # Save checkpoints
    df.to_csv(f'updated_{dataset}_df.csv', index=False)
    with open(f'data_dict_{dataset}.pickle', 'wb') as handle:
        pickle.dump(data_dicts[dataset], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
process_files_sequentially('train')

In [ ]:
process_files_sequentially('dev')

In [ ]:
process_files_sequentially('test')

In [5]:
#@title Restore checkpoints after processing
def load_data(dataset):
    df = pd.read_csv(f'updated_{dataset}_df.csv')
    with open(f'data_dict_{dataset}.pickle', 'rb') as handle:
        data_dict = pickle.load(handle)
    return df, data_dict

train_df, data_dict_train = load_data('train')
dev_df, data_dict_dev = load_data('dev')
test_df, data_dict_test = load_data('test')

data_dicts = {'train': data_dict_train, 'dev': data_dict_dev, 'test': data_dict_test}
dataframes = { 'train': {'df':train_df, 'folder':'train_splits'}, 'dev': {'df':dev_df, 'folder':'dev_splits_complete'}, 'test': {'df': test_df, 'folder': 'output_repeated_splits_test'}}

In [46]:
print(data_dicts['train'][12]['text']['input_ids'].shape)

torch.Size([1, 256])


In [45]:
print(data_dicts['train'][12]['audio'].shape)

torch.Size([192])


In [44]:
text_dim = 768 # Because BERT
# audio_dim = data_dicts['train'][1]['audio'].shape[-1]
audio_dim = 192 # Because speechbrain ECAPA-TDNN

print("text_dim: ", text_dim, "\naudio_dim: ", audio_dim)

text_dim:  768 
audio_dim:  192


## Contrastive Learning

1. Modality Fusion: Use an attention mechanism to combine the audio and text features for each data instance.

2. Contrastive Learning: Use a Contrastive Learning approach to train a model that can distinguish between instances with the same emotion (positive pairs) and instances with different emotions (negative pairs).

3. Model Architecture.

4. Loss Function and Training

In [51]:
class Attention(nn.Module):
    def __init__(self, in_features):
        super(Attention, self).__init__()
        self.linear = nn.Linear(in_features, 1)

    def forward(self, x):
        scores = self.linear(x)
        scores = F.softmax(scores, dim=-2) 
        weighted_sum = scores * x
        return torch.sum(weighted_sum, dim=-2)


class MultimodalModel(nn.Module):
    def __init__(self, audio_dim, text_dim=768):  # Update default text_dim to 768 for BERT embeddings
        super(MultimodalModel, self).__init__()
        self.audio_attention = Attention(audio_dim)
        self.text_attention = Attention(text_dim)
        self.fc = nn.Linear(audio_dim+text_dim, 256)

    def forward(self, audio, text):
        audio_repr = self.audio_attention(audio)
        text_repr = self.text_attention(text)
        combined = torch.cat((audio_repr, text_repr), dim=1)
        out = self.fc(combined)
        return F.normalize(out, dim=1)

In [48]:
#@title Contrastive Loss Function
class TripletMarginLoss(nn.Module):
    def __init__(self, margin):
        super(TripletMarginLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        pos_dist = F.pairwise_distance(anchor, positive)
        neg_dist = F.pairwise_distance(anchor, negative)
        loss = F.relu(pos_dist - neg_dist + self.margin)
        return loss.mean()

In [52]:
class ContrastiveLearningDataset(Dataset):
    def __init__(self, data_list):
        self.data_list = data_list
        self.labels = np.array([item['emotion'] for item in data_list])

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        anchor = self.data_list[idx]
        positive_idx = np.random.choice(np.where(self.labels == anchor['emotion'])[0])
        negative_idx = np.random.choice(np.where(self.labels != anchor['emotion'])[0])
        positive = self.data_list[positive_idx]
        negative = self.data_list[negative_idx]

        return (anchor['audio'].unsqueeze(0), anchor['text']['input_ids'].squeeze(-1)), anchor['emotion'], \
               (positive['audio'].unsqueeze(0), positive['text']['input_ids'].squeeze(-1)), \
               (negative['audio'].unsqueeze(0), negative['text']['input_ids'].squeeze(-1))




data_lists = {dataset: list(data_dicts[dataset].values()) for dataset in ['train', 'dev', 'test']}

data_loaders = {dataset: DataLoader(ContrastiveLearningDataset(data_lists[dataset]), batch_size=32, shuffle=True, num_workers=num_cores) for dataset in ['train', 'dev', 'test']}

In [27]:
from transformers import AutoModel

bert_model = AutoModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
bert_model.to(device)

In [66]:
import optuna

print_every = 5

best_results = {'loss': float('inf'), 'model': None}

def objective(trial, resume=False, num_epochs=1):
    
    contrastive_model = MultimodalModel(audio_dim=audio_dim, text_dim=text_dim)
    if resume:
        contrastive_model.load_state_dict(torch.load('contrastive_model.pt'))
    contrastive_model.to(device)

    # Define optimizer
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1) if not resume else study.best_params['lr']
    optimizer = optim.Adam(contrastive_model.parameters(), lr=lr)

    # Define loss function
    margin = trial.suggest_uniform('margin', 0.1, 1.0) if not resume else study.best_params['margin']
    criterion = TripletMarginLoss(margin=margin)

    # Training loop
    for epoch in range(num_epochs):
        contrastive_model.train()
        running_loss = 0.0
        for i, data in tqdm(enumerate(data_loaders['train']), total=len(data_loaders['train'])):
            (anchor_audio, anchor_text), anchor_label, (positive_audio, positive_text), (negative_audio, negative_text) = data

            # Move tensors to the right device
            anchor_audio = anchor_audio.to(device)
            anchor_text = anchor_text.to(device)
            positive_audio = positive_audio.to(device)
            positive_text = positive_text.to(device)
            negative_audio = negative_audio.to(device)
            negative_text = negative_text.to(device)

            with torch.no_grad():
                anchor_text = bert_model(anchor_text.squeeze(1))[0]
                positive_text = bert_model(positive_text.squeeze(1))[0]
                negative_text = bert_model(negative_text.squeeze(1))[0]

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            anchor_out = contrastive_model(anchor_audio, anchor_text)
            positive_out = contrastive_model(positive_audio, positive_text)
            negative_out = contrastive_model(negative_audio, negative_text)
            loss = criterion(anchor_out, positive_out, negative_out)
            loss.backward()
            optimizer.step()

            # Print statistics
            running_loss += loss.item()
        if epoch % print_every == 0:
            print('[%d] loss: %.3f' % (epoch + 1, running_loss / print_every))
            running_loss = 0.0

        # Validation
        print(teal, "Now running validation...")
        contrastive_model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for i, data in tqdm(enumerate(data_loaders['dev']), total=len(data_loaders['dev'])):
                (anchor_audio, anchor_text), anchor_label, (positive_audio, positive_text), (negative_audio, negative_text) = data

                # Move tensors to the right device
                anchor_audio = anchor_audio.to(device)
                anchor_text = anchor_text.to(device)
                positive_audio = positive_audio.to(device)
                positive_text = positive_text.to(device)
                negative_audio = negative_audio.to(device)
                negative_text = negative_text.to(device)

                with torch.no_grad():
                    anchor_text = bert_model(anchor_text.squeeze(1))[0]
                    positive_text = bert_model(positive_text.squeeze(1))[0]
                    negative_text = bert_model(negative_text.squeeze(1))[0]

                anchor_out = contrastive_model(anchor_audio, anchor_text)
                positive_out = contrastive_model(positive_audio, positive_text)
                negative_out = contrastive_model(negative_audio, negative_text)
                loss = criterion(anchor_out, positive_out, negative_out)
                val_loss += loss.item()

        if val_loss < best_results['loss']:            
            print(blue, "New best validation loss achieved! ", val_loss) 
            print(blue, "Best margin ", margin)
            print(blue, "Best lr ", lr)
            best_results['loss'] = val_loss
            best_results['model'] = contrastive_model.state_dict()
            torch.save(best_results['model'], 'contrastive_model.pt')
        if not resume and trial.should_prune():
            raise optuna.TrialPruned()
            
    return best_results['loss']

study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=30)

best_pretrained_params = study.best_params
with open('best_pretrained_params.pickle', 'wb') as handle:
    pickle.dump(best_pretrained_params, handle, protocol=pickle.HIGHEST_PROTOCOL)

objective(None, resume=True, num_epochs=30)

[I 2023-05-03 18:28:33,468] A new study created in memory with name: no-name-54eb54cb-1789-45a8-8ad8-b1e39202cb7e
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
100%|██████████| 261/261 [01:53<00:00,  2.30it/s]

[1] loss: 35.993
 Now running validation...



100%|██████████| 30/30 [00:12<00:00,  2.36it/s]
[I 2023-05-03 18:30:40,367] Trial 0 finished with value: 20.28972750902176 and parameters: {'lr': 0.00011440199476855864, 'margin': 0.6993460620691053}. Best is trial 0 with value: 20.28972750902176.


 New best validation loss achieved!  20.28972750902176
 Best margin  0.6993460620691053
 Best lr  0.00011440199476855864


100%|██████████| 261/261 [01:53<00:00,  2.30it/s]

...

 Now running validation...



100%|██████████| 261/261 [01:53<00:00,  2.29it/s]

 Now running validation...



100%|██████████| 30/30 [00:12<00:00,  2.36it/s]


2.4147740341722965

In [68]:
#@title Evaluate Contrastive Model on test set

def evaluate_model(model, criterion, data_loader):
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for i, data in tqdm(enumerate(data_loader), total=len(data_loader)):
            (anchor_audio, anchor_text), anchor_label, (positive_audio, positive_text), (negative_audio, negative_text) = data

            anchor_audio = anchor_audio.to(device)
            anchor_text = anchor_text.to(device)
            positive_audio = positive_audio.to(device)
            positive_text = positive_text.to(device)
            negative_audio = negative_audio.to(device)
            negative_text = negative_text.to(device)
            
            with torch.no_grad():
                anchor_text = bert_model(anchor_text.squeeze(1))[0]
                positive_text = bert_model(positive_text.squeeze(1))[0]
                negative_text = bert_model(negative_text.squeeze(1))[0]

            anchor_out = model(anchor_audio, anchor_text)
            positive_out = model(positive_audio, positive_text)
            negative_out = model(negative_audio, negative_text)
            loss = criterion(anchor_out, positive_out, negative_out)
            test_loss += loss.item()

    return test_loss / len(data_loader)

# Assuming the best model parameters are found and loaded into the model
contrastive_model = MultimodalModel(audio_dim=audio_dim, text_dim=text_dim).to(device)
contrastive_model.load_state_dict(torch.load('contrastive_model.pt'))

# Create the loss function
with open('best_pretrained_params.pickle', 'rb') as handle:
    best_pretrained_params = pickle.load(handle)
margin = best_pretrained_params['margin']
criterion = TripletMarginLoss(margin=margin)

# Evaluate the model
test_loss = evaluate_model(contrastive_model, criterion, data_loaders['test'])
print('Test Loss: %.3f' % test_loss)

100%|██████████| 68/68 [00:29<00:00,  2.32it/s]

Test Loss: 0.094


## Downstream Task Training

Fine-tuning the contrastive pre-trained model to predict emotion and sentiment

In [ ]:
class DownstreamTaskDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        audio = self.dataset[idx]['audio'].unsqueeze(0)
        text = self.dataset[idx]['text']['input_ids'].squeeze(-1)
        sentiment = self.dataset[idx]['sentiment']
        emotion = self.dataset[idx]['emotion']

        return (audio, text), sentiment, emotion

In [ ]:
batch_size = 16

train_dataset = DownstreamTaskDataset(data_dicts['train'])
dev_dataset = DownstreamTaskDataset(data_dicts['dev'])
test_dataset = DownstreamTaskDataset(data_dicts['test'])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

data_loaders = {
    'train': train_dataloader,
    'dev': dev_dataloader,
    'test': test_dataloader
}

In [ ]:
class DownstreamModel(nn.Module):
    def __init__(self, contrastive_model, num_sentiment_classes, num_emotion_classes, dropout_rate):
        super(DownstreamModel, self).__init__()
        self.contrastive_model = contrastive_model
        self.dropout = nn.Dropout(dropout_rate)
        self.sentiment_fc = nn.Linear(256, num_sentiment_classes)
        self.emotion_fc = nn.Linear(256, num_emotion_classes)

    def forward(self, audio, text):
        x = self.contrastive_model(audio, text)
        x = self.dropout(x)
        sentiment_out = self.sentiment_fc(x)
        emotion_out = self.emotion_fc(x)
        return sentiment_out, emotion_out

In [ ]:
#@title Training the Final Models

num_epochs = 15
print_every = 5

def objective(trial):
    dropout_rate = trial.suggest_uniform('dropout_rate', 0, 0.5)
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    sentiment_loss_weight = trial.suggest_uniform('sentiment_loss_weight', 0.1, 1.0)
    emotion_loss_weight = trial.suggest_uniform('emotion_loss_weight', 0.1, 1.0)

    multimodal_model = DownstreamModel(contrastive_model=contrastive_model, num_sentiment_classes=3, num_emotion_classes=7, dropout_rate).to(device)

    optimizer = optim.Adam(multimodal_model.parameters(), lr=lr, weight_decay=1e-5)

    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        multimodal_model.train()
        running_loss = 0.0
        for i, (inputs, sentiment_label, emotion_label) in tqdm(enumerate(data_loaders['train'], 0)):
            inputs = tuple(input.to(device) for input in inputs)
            inputs[1] = bert_model(inputs[1].squeeze(1))[0] # Text input through BERT

            optimizer.zero_grad()

            sentiment_out, emotion_out = multimodal_model(*inputs)
            sentiment_loss = criterion(sentiment_out, sentiment_label.to(device))
            emotion_loss = criterion(emotion_out, emotion_label.to(device))
            loss = sentiment_loss_weight * sentiment_loss + emotion_loss_weight * emotion_loss
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix({'loss': running_loss / (i+1)})
        if epoch % print_every == 0:
            print('[%d] loss: %.3f' % (epoch + 1, running_loss / print_every))
            running_loss = 0.0

    multimodal_model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, (inputs, sentiment_label, emotion_label) in enumerate(data_loaders['dev'], 0):
            inputs = tuple(input.to(device) for input in inputs)

            sentiment_out, emotion_out = multimodal_model(*inputs)
            sentiment_loss = criterion(sentiment_out, sentiment_label.to(device))
            emotion_loss = criterion(emotion_out, emotion_label.to(device))
            loss = sentiment_loss_weight * sentiment_loss + emotion_loss_weight * emotion_loss
            val_loss += loss.item()

    if trial.value == study.best_value:
        torch.save(multimodal_model.state_dict(), 'best_model.pth')
    return val_loss / len(data_loaders['dev'])

# Load pre-trained model
if (model is not None):
  contrastive_model = model
else:
  contrastive_model = MultimodalModel(audio_dim=audio_dim, text_dim=text_dim).to(device)
  contrastive_model.load_state_dict(torch.load('contrastive_model.pth'))

study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=50)

print(green, "Best trial:")
trial = study.best_trial
print(f"Value: {trial.value}")
print("Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Save the best parameters
best_params = study.best_params
with open('best_params.pickle', 'wb') as handle:
    pickle.dump(best_params, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#@title Evaluate Model

from sklearn.metrics import classification_report

# Initialize lists to hold labels and predictions
sentiment_labels = []
sentiment_preds = []
emotion_labels = []
emotion_preds = []

with open('best_params.pickle', 'rb') as handle:
    best_params = pickle.load(handle)
dropout_rate = best_params['dropout_rate']

# Load pre-trained model
if contrastive_model is None:
  contrastive_model = MultimodalModel(audio_dim=audio_dim, text_dim=text_dim).to(device)
  contrastive_model.load_state_dict(torch.load('contrastive_model.pth'))

multimodal_model = DownstreamModel(contrastive_model=contrastive_model, 
                                   num_sentiment_classes=3, 
                                   num_emotion_classes=7, 
                                   dropout_rate=dropout_rate).to(device)

multimodal_model.load_state_dict(torch.load('best_model.pth'))

multimodal_model.eval()

total = 0
sentiment_correct = 0
emotion_correct = 0

with torch.no_grad():
    for i, (inputs, sentiment_label, emotion_label) in enumerate(data_loaders['test'], 0):
        inputs = tuple(input.to(device) for input in inputs)
        
        sentiment_out, emotion_out = multimodal_model(*inputs)
        _, sentiment_predicted = torch.max(sentiment_out.data, 1)
        _, emotion_predicted = torch.max(emotion_out.data, 1)
        total += sentiment_label.size(0)
        sentiment_correct += (sentiment_predicted == sentiment_label.to(device)).sum().item()
        emotion_correct += (emotion_predicted == emotion_label.to(device)).sum().item()
        sentiment_labels.extend(sentiment_label.cpu().numpy())
        sentiment_preds.extend(sentiment_predicted.cpu().numpy())
        emotion_labels.extend(emotion_label.cpu().numpy())
        emotion_preds.extend(emotion_predicted.cpu().numpy())

print(green, 'Accuracy of the sentiment prediction on the test data: %d %%' % (100 * sentiment_correct / total))
print('Accuracy of the emotion prediction on the test data: %d %%' % (100 * emotion_correct / total))

print('Classification report for sentiment prediction:')
print(classification_report(sentiment_labels, sentiment_preds))

print('Classification report for emotion prediction:')
print(classification_report(emotion_labels, emotion_preds))